<font size = "5">

Momentum Trading Strategy

In [27]:
import pandas as pd
import numpy as np

<font size = "4">

Step 0: Collect our data from our priceData function. For this walkthrough we will use ``AAPL`` Apple as an example.

In [28]:
from quantlab.data.yf import priceData
from quantlab.data.tickers import tickers

data = priceData(tickers, 150)
s = pd.Series(data["AAPL"])

[*********************100%***********************]  239 of 239 completed

25 Failed downloads:
['SYK']: Timeout('Failed to perform, curl: (28) Connection timed out after 10003 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['LYFT', 'VICI', 'MET', 'RCL', 'NVS', 'ROKU', 'HIG', 'BLK', 'MRK', 'ASML', 'AFG', 'MPW', 'LMT', 'DOW', 'ADP', 'INVH', 'PLTR', 'TFC', 'ADBE', 'DHR']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['BKNG', 'BEN', 'BX', 'VNO']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


<font size = "4">

Step 1: Find the ``current_price`` of the stock as well as the ``past_price`` from 8 trading days ago, and then log and divide to find the logarithmic ``change``. Logarithmic allows us to check for proportional changes instead of linear ones.

In [29]:
current_price = s.iloc[-1]
past_price = s.iloc[-8]
change = np.log(current_price / past_price)

In [30]:
print(f"current_price: {current_price}")
print(f"past_price: {past_price}")
print(f"change: {change}")


current_price: 269.0
past_price: 252.2899932861328
change: 0.06413218670677084


<font size = "4">

Step 2: Find the volatility ``vol`` by looking at te standard deviation of the ``daily_returns`` over the past 8 days. Create a ``threshold`` of 0.75 times the volatility times the square root of our 8 day lookback period.

In [31]:
daily_returns = np.log(s/s.shift(1))
vol = daily_returns.iloc[-8:].std(ddof = 1)
threshold = 0.75 * vol * np.sqrt(8)

In [32]:
print(f"daily_returns: {np.array(daily_returns[-5:])} ...")
print(f"vol: {vol}")
print(f"threshold: {threshold}")

daily_returns: [-0.01657678  0.00436259  0.01240453  0.02253539  0.00070658] ...
vol: 0.01673301365000057
threshold: 0.035496082264807396


<font size = "4">

Step 3: If our change in price over the past 8 days is over the threshold, the stock is believed to have sufficient positive momentum to trigger our signal, and we long the stock. If the change in price is less than the negative of the threshold, the stock is believed to have sufficient negative momentum, and we short the stock. The amount we buy should be standardized by price, so we divide 100 by the price of a share in order to buy/sell $100 of each stock.

In [33]:
if change > threshold:
    result = 100 / current_price
    print("Since the change is greater than the threshold, momentum is high, long the stock.")
elif change < -threshold:
    result = -100 / current_price
    print("Since the change is less than the negative of the threshold, momentum is high, short the stock.")
else:
    print("Since the change is within the positive and negative threshold, do nothing.")

Since the change is greater than the threshold, momentum is high, long the stock.


<font size = "4">

Now, Wrap all of this as a function, looping across all columns. Start by initializing an empty data frame ``results`` , which will be added to if our result consists of us buying or selling a stock. Return the data frame of trading decisions of all stocks.

In [34]:
def momentum(data):
    results = {}
    for col in data.columns:
        s = pd.Series(data[col])
        current_price = s.iloc[-1]
        past_price = s.iloc[-8]
        change = np.log(current_price / past_price)
        daily_returns = np.log(s/s.shift(1))
        vol = daily_returns.iloc[-8:].std(ddof = 1)
        threshold = 0.75 * vol * np.sqrt(8)

        if change > threshold:
            results[col] = 100 / current_price
        elif change < -threshold:
            results[col] = -100 / current_price
        else:
            continue
    return pd.Series(results, name = "shares")

In [35]:
results = momentum(data)
print(results)

A       0.685636
AAPL    0.371747
AEP    -0.868734
AJG    -0.373832
ALL    -0.523972
          ...   
WFC     1.150748
WPC    -1.512630
XEL    -1.252819
XOM     0.869338
ZM      1.177995
Name: shares, Length: 102, dtype: float64
